In [1]:
from qslamr import *
from model_design import INITIALDICT, GRIDDICT

In [2]:
positions = GRIDDICT.values()
test = ParticleFilter(positions, **INITIALDICT)
measurement_controls = [[0, 0], [0, 2]]*2 # an upstate on Qubit 1 and Qubit 3

In [3]:
test.qslamr(measurement_controls)

Initial alpha weights (below) are uniformly distributed...
[ 0.33333333  0.33333333  0.33333333]
and alpha_weights sum to 1:
1.0

in likelihood_func_alpha, msmt_dj -10.0
in likelihood_func_alpha, prob_j -10.0
in likelihood_func_alpha, var_r 1.0
in likelihood_func_alpha, rho_0 0.368746380373


in alpha_weight_calc, old_weight 0.333333333333
in alpha_weight_calc, likelihood 0.184373190186


in likelihood_func_alpha, msmt_dj -10.0
in likelihood_func_alpha, prob_j -10.0
in likelihood_func_alpha, var_r 1.0
in likelihood_func_alpha, rho_0 0.368746380373


in alpha_weight_calc, old_weight 0.333333333333
in alpha_weight_calc, likelihood 0.184373190186


in likelihood_func_alpha, msmt_dj -10.0
in likelihood_func_alpha, prob_j -10.0
in likelihood_func_alpha, var_r 1.0
in likelihood_func_alpha, rho_0 0.368746380373


in alpha_weight_calc, old_weight 0.333333333333
in alpha_weight_calc, likelihood 0.184373190186

raw_weights in calc_weights_set [ 0.06145773  0.06145773  0.06145773]

In generate_be

AttributeError: 'NoneType' object has no attribute 'particles'

In [12]:
print test.AlphaSet.particles[3].BetaAlphaSet_j

IndexError: list index out of range

#### Test unpacking of a slice of an array into elements

In [ ]:
nodes = 3
statetype = 4
a = range(nodes*statetype)

In [ ]:
a[0::nodes]

In [ ]:
a[1::nodes]

In [ ]:
x, y, z, r = a[2::nodes]

#### Test the flattening of lists into arrays - cases of success and failure

In [ ]:
x = [[3.4, 2.0],[2, 1]]
y = [[3.4],[2, 1]]
z = [3.4,[2, 1]]

In [ ]:
np.asarray(x).flatten()

In [ ]:
np.asarray(y).flatten()

In [ ]:
np.asarray(z).flatten()